In [ ]:
import xarray as xr
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import scipy

In [ ]:
fnames=['ORCA025-SE-NEMO_1990_2019_GS1p2_full_SST_SSS_PEA_MonClimate.nc',
'ORCA025-SE-NEMO_1990_2019_GS1p0_notide_SST_SSS_PEA_MonClimate.nc']
names=['GS1p7_full','GS1p0_notide']

dpath='/home/users/jholt/work/SENEMO/ASSESSMENT/ORCA025-SE-NEMO'
fn_bathymetry='/home/users/jholt/work/SENEMO/senemo/INPUTS/eORCA025_bathy_meter.nc' 

datapath_LME ='/home/users/jholt/work/SENEMO/ASSESSMENT/EN4/'
DATANAME='ORCA025_1978-2019'
DATANAME='ORCA025'
f_bathy=xr.open_dataset(fn_bathymetry)
J_offset=186 #account for extra rows in eORCA if data is made for normal ORCA
A=np.load('../Data/LME_gridinfo_V4.npz')
a=scipy.io.loadmat('../Data/ORCA025_ROAM_GLB_LMEmaskV4.mat')
LME_mask=a['LME_mask'][:,:].T

In [ ]:
LME_Clusters='../Data/LME_Clusters_eORCA025.csv'
clusters = pd.read_csv(LME_Clusters)
nclusters=clusters.values.shape[0]
iwant_clusters=range(23)

In [ ]:
#ls /home/users/jholt/work/SENEMO/ASSESSMENT/EN4/ORCA025/
 ls /home/users/jholt/work/SENEMO/ASSESSMENT/EN4//ORCA025/Laptev_Sea_57_EN4_eORCA025_PEA_SST_SSS_V3.nc

In [ ]:
SST_EN4={}
SSS_EN4={}
PEA_EN4={}

for icluster in iwant_clusters:

     lmes=clusters.values[icluster,10:]
     ilmes=np.array([])
     for LME in lmes:
     #try:
     #    ilmes=np.append(ilmes,int(LME[-2:]))
     #except:
     #    ilmes=np.append(ilmes,-1)
        if not type(LME)==float:
             ilmes=np.append(ilmes,int(LME[-2:]))
        else:
             ilmes=np.append(ilmes,-1)

    
    
             
     Cluster_Name=clusters.values[icluster,1]
     #eORCA025 index of cluster
     i_min1=int(clusters.values[icluster,2])
     i_max1=int(clusters.values[icluster,3])
     j_min1=int(clusters.values[icluster,4])
     j_max1=int(clusters.values[icluster,5])
     
     D=f_bathy.Bathymetry.values[j_min1:j_max1+1,i_min1:i_max1+1]
     Depth=np.repeat(D[np.newaxis,:,:],12,axis=0)
     SST_EN4[icluster]=np.zeros(Depth.shape)
     SSS_EN4[icluster]=np.zeros(Depth.shape)
     PEA_EN4[icluster]=np.zeros(Depth.shape)     
     mask=np.zeros((Depth.shape[1],Depth.shape[2]))
     for ilme,LMENAM in enumerate(lmes):
        if isinstance(LMENAM,str):
                            
            LMEmask=LME_mask[(j_min1-J_offset):(j_max1-J_offset+1),i_min1:i_max1+1]
       # try:
            #EN4=xr.open_dataset(f"{datapath_LME}/{DATANAME}/{LMENAM}_1978_2019_EN4_eORCA025_PEA_SST_SSS_v1.nc")
            is_file=True
            try:
                EN4=xr.open_dataset(f"{datapath_LME}/{DATANAME}/{LMENAM}_EN4mnthgrid_V3.nc")
            except:
                is_file=False
                print(f"{datapath_LME}/{DATANAME}/{LMENAM}_EN4_eORCA025_PEA_SST_SSS_V3.nc")
            if is_file:
                #ORCA025 index of LME bounds
                lims=EN4.lims.values
                #lims=np.array([lims[2],lims[3],lims[0],lims[1]]) # re-order to i,i,j,j
                #conver to eORCA grid
                lims[[2,3]]=lims[[2,3]] +J_offset

                lims1=np.copy(lims)
                lims1[0]=np.max([lims[0],i_min1])
                lims1[1]=np.min([lims[1],i_max1])
                lims1[2]=np.max([lims[2],j_min1])
                lims1[3]=np.min([lims[3],j_max1])                
                #index of the LME data
                i_min2=lims1[0]-lims[0]
                i_max2=lims1[1]-lims[0]
                j_min2=lims1[2]-lims[2]
                j_max2=lims1[3]-lims[2]                
                #Index of LME portion of CLuster
                i_min=lims1[0]-i_min1
                i_max=lims1[1]-i_min1
                j_min=lims1[2]-j_min1
                j_max=lims1[3]-j_min1
                mask[LMEmask==ilmes[ilme]]=1
                #SST_EN4_LME=EN4.sst_g.values[:,:,:]
                #SSS_EN4_LME=EN4.sss_g.values[:,:,:]
                #PEA_EN4_LME=EN4.pea_g.values[:,:,:]
                SST_EN4_LME=EN4.SST_g.values[:,:,:].T
                SSS_EN4_LME=EN4.SSS_g.values[:,:,:].T
                PEA_EN4_LME=EN4.PEA_g.values[:,:,:].T
                
                SST_EN4_LME[~np.isfinite(SST_EN4_LME)]=0.
                SSS_EN4_LME[~np.isfinite(SSS_EN4_LME)]=0.
                PEA_EN4_LME[~np.isfinite(PEA_EN4_LME)]=0.
                PEA_EN4_LME[PEA_EN4_LME<0]=0.    
                PEA_EN4_LME[PEA_EN4_LME>1000]=0.    
                mm=LMEmask[j_min:j_max+1,i_min:i_max+1]==ilmes[ilme]
                #mm=1
                SST_EN4[icluster][:,j_min:j_max+1,i_min:i_max+1]=(SST_EN4[icluster][:,j_min:j_max+1,i_min:i_max+1]+
                                                                 SST_EN4_LME[:,j_min2:j_max2+1,i_min2:i_max2+1]*mm)
                SSS_EN4[icluster][:,j_min:j_max+1,i_min:i_max+1]=(SSS_EN4[icluster][:,j_min:j_max+1,i_min:i_max+1]+
                                                                  SSS_EN4_LME[:,j_min2:j_max2+1,i_min2:i_max2+1]*mm)
                PEA_EN4[icluster][:,j_min:j_max+1,i_min:i_max+1]=(PEA_EN4[icluster][:,j_min:j_max+1,i_min:i_max+1]+
                                                                  PEA_EN4_LME[:,j_min2:j_max2+1,i_min2:i_max2+1]*mm)
           # except:
           #     continue
    
             
     mask=np.repeat(mask[np.newaxis,:,:],12,axis=0)
    
     SST_EN4[icluster][SST_EN4[icluster]==0]=np.nan
     SSS_EN4[icluster][SSS_EN4[icluster]==0]=np.nan
     PEA_EN4[icluster][PEA_EN4[icluster]==0]=np.nan
     
     SST_EN4[icluster][mask==0]=np.nan
     SSS_EN4[icluster][mask==0]=np.nan
     PEA_EN4[icluster][mask==0]=np.nan

In [ ]:
plt.pcolormesh(PEA_EN4[11][6,:,:]);plt.colorbar(orientation='horizontal')


In [ ]:
PEA_co_mean={}
SSS_co_mean={}
SST_co_mean={}

for iname,fname in enumerate(fnames):
    name=names[iname]
    f=xr.open_dataset(f"{dpath}/{fname}")
    for icluster in range(23):
        print(clusters.values[icluster,1])
        i_min1=int(clusters.values[icluster,2])
        i_max1=int(clusters.values[icluster,3])
        j_min1=int(clusters.values[icluster,4])
        j_max1=int(clusters.values[icluster,5])
        D=f_bathy.Bathymetry.values[j_min1:j_max1+1,i_min1:i_max1+1]
        PEAy=f.pea_monthy_clim.values[:,j_min1:j_max1+1,i_min1:i_max1+1]
        PEA_co=np.copy(PEAy)
        PEA_co[np.isnan(PEA_EN4[icluster])]=np.nan

        SSSy=f.sss_monthy_clim.values[:,j_min1:j_max1+1,i_min1:i_max1+1]
        SSS_co=np.copy(SSSy)
        SSS_co[np.isnan(SSS_EN4[icluster])]=np.nan

        SSTy=f.sst_monthy_clim.values[:,j_min1:j_max1+1,i_min1:i_max1+1]
        SST_co=np.copy(SSTy)
        SST_co[np.isnan(SST_EN4[icluster])]=np.nan
        
        DD=np.repeat(D[np.newaxis,:,:],12,axis=0)
        DD.shape
        PEA_co[DD>200]=np.nan
        PEA_co_mean[name,icluster]=np.zeros(12)
        for it in range(12):
            PEA_co_mean[name,icluster][it]=np.nanmean(PEA_co[it,:,:])

        SSS_co[DD>200]=np.nan
        SSS_co_mean[name,icluster]=np.zeros(12)
        for it in range(12):
            SSS_co_mean[name,icluster][it]=np.nanmean(SSS_co[it,:,:])
        
        SST_co[DD>200]=np.nan  
        SST_co_mean[name,icluster]=np.zeros(12)
        for it in range(12):
            SST_co_mean[name,icluster][it]=np.nanmean(SST_co[it,:,:])


In [ ]:
PEA_EN4_co_mean={}
SSS_EN4_co_mean={}
SST_EN4_co_mean={}

for icluster in range(23):

    i_min1=int(clusters.values[icluster,2])
    i_max1=int(clusters.values[icluster,3])
    j_min1=int(clusters.values[icluster,4])
    j_max1=int(clusters.values[icluster,5])
    D=f_bathy.Bathymetry.values[j_min1:j_max1+1,i_min1:i_max1+1]
    
    PEA_co=np.copy(PEA_EN4[icluster])
    SSS_co=np.copy(SSS_EN4[icluster])
    SST_co=np.copy(SST_EN4[icluster])
    
    DD=np.repeat(D[np.newaxis,:,:],12,axis=0)

    PEA_co[DD>200]=np.nan
    SSS_co[DD>200]=np.nan
    SST_co[DD>200]=np.nan
    
    PEA_EN4_co_mean[icluster]=np.zeros(12)
    SSS_EN4_co_mean[icluster]=np.zeros(12)
    SST_EN4_co_mean[icluster]=np.zeros(12)
    
    for it in range(12):
        PEA_EN4_co_mean[icluster][it]=np.nanmean(PEA_co[it,:,:])
        SSS_EN4_co_mean[icluster][it]=np.nanmean(SSS_co[it,:,:])
        SST_EN4_co_mean[icluster][it]=np.nanmean(SST_co[it,:,:])
        
    if icluster==11:
        plt.figure(figsize=(12,10))
        plt.pcolormesh(SSS_EN4[11][0,:,:]);plt.colorbar(orientation='horizontal')
        #plt.hist(PEA_EN4[11][0,:,:].ravel(),100)

In [ ]:
t=range(12)
fig,ax=plt.subplots(6,4,sharex=True,figsize=(12,16))
ax=ax.ravel()
for icluster in range(23):
    ax[icluster].plot(t,PEA_co_mean[names[0],icluster],t,PEA_co_mean[names[1],icluster],t,PEA_EN4_co_mean[icluster],'x')
    ax[icluster].set_title(f"PEA {clusters.values[icluster,1]}")
    ax[icluster].set_xticks(range(12))    
    ax[icluster].set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])

ax[0].legend(['GS1p7','GS1p0','EN4'])

plt.savefig('../Figures/PEA_ann_cyc_clusters_GS1p7_GS1p0_v2.png',dpi=300,facecolor='white')


In [ ]:
fig,ax=plt.subplots(6,4,sharex=True,figsize=(12,16))
ax=ax.ravel()
for icluster in range(23):
    ax[icluster].plot(t,SSS_co_mean[names[0],icluster],t,SSS_co_mean[names[1],icluster],t,SSS_EN4_co_mean[icluster],'x')
    ax[icluster].set_title(f"SSS {clusters.values[icluster,1]}")
    ax[icluster].set_xticks(range(12))    
    ax[icluster].set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])

ax[0].legend(['GS1p7','GS1p0','EN4'])
plt.savefig('../Figures/SSS_ann_cyc_clusters_GS1p7_GS1p0_v2.png',dpi=300,facecolor='white')

In [ ]:
fig,ax=plt.subplots(6,4,sharex=True,figsize=(12,16))
ax=ax.ravel()
for icluster in range(23):
    ax[icluster].plot(t,SST_co_mean[names[0],icluster],t,SST_co_mean[names[1],icluster],t,SST_EN4_co_mean[icluster],'x')
    ax[icluster].set_title(f"SST {clusters.values[icluster,1]}")
    ax[icluster].set_xticks(range(12))    
    ax[icluster].set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])

ax[0].legend(['GS1p7','GS1p0','EN4'])
plt.savefig('../Figures/SST_ann_cyc_clusters_GS1p7_GS1p0_v2.png',dpi=300,facecolor='white')